In [1]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import IPython.display
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [2]:
os.chdir('/Users/krzysztofwasniewski/Desktop/UK Electricity')

In [3]:
os.listdir()

['Electricity_since_1920.xlsx',
 'fes2023_es2_v001.csv',
 '.DS_Store',
 'DemandData Field Descriptions_1.doc',
 '202402290812_Embedded_Forecast.csv',
 'Demand data 2022.xlsx',
 'NETS SQSS draft v2.7 awaiting licence approval from Ofgem_1.pdf',
 '~$Demand 2022 Pearson.xlsx',
 'UK Demand Data 2022 normalized label Month.docx',
 'UK data 2022 normalized label Settlement Period.docx',
 'System Operating Plan as for March 2024 model summary.docx',
 'UK Data 2021 normalized label Settlement Period.docx',
 'UK 2021 normalized label settlement period.docx',
 'DemandData_2021_1690195771550.csv',
 'UK Demand stats 2022.xlsx',
 'Demand 2022 Pearson.xlsx',
 'UK Data 2021 normalized label Month.docx',
 'Links from Michael Short.docx',
 'UK Demand stats 2021.xlsx',
 'glossary-of-terms-used-in-system-operating-plan_final.pdf',
 'UK data 2022 normalized label Month.docx',
 'FAQ-2.doc',
 'DemandDataUpdate.csv',
 'DemandData_2020_1690195756332.csv',
 'DemandData_2022_1690195791146.csv',
 'Demand data 20

In [4]:
Emb_For_20240229=pd.DataFrame(pd.read_csv('202402290812_Embedded_Forecast.csv'))

In [5]:
Emb_For_20240229.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DATE_GMT                 608 non-null    int64 
 1   TIME_GMT                 608 non-null    int64 
 2   SETTLEMENT_DATE          608 non-null    object
 3   SETTLEMENT_PERIOD        608 non-null    int64 
 4   EMBEDDED_WIND_FORECAST   608 non-null    int64 
 5   EMBEDDED_WIND_CAPACITY   608 non-null    int64 
 6   EMBEDDED_SOLAR_FORECAST  608 non-null    int64 
 7   EMBEDDED_SOLAR_CAPACITY  608 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 38.1+ KB


In [6]:
Dem_Update=pd.DataFrame(pd.read_csv('DemandDataUpdate.csv'))
Dem_Update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3216 entries, 0 to 3215
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   SETTLEMENT_DATE            3216 non-null   object
 1   SETTLEMENT_PERIOD          3216 non-null   int64 
 2   ND                         3216 non-null   int64 
 3   FORECAST_ACTUAL_INDICATOR  3216 non-null   object
 4   TSD                        3216 non-null   int64 
 5   ENGLAND_WALES_DEMAND       3216 non-null   int64 
 6   EMBEDDED_WIND_GENERATION   3216 non-null   int64 
 7   EMBEDDED_WIND_CAPACITY     3216 non-null   int64 
 8   EMBEDDED_SOLAR_GENERATION  3216 non-null   int64 
 9   EMBEDDED_SOLAR_CAPACITY    3216 non-null   int64 
 10  NON_BM_STOR                3216 non-null   int64 
 11  PUMP_STORAGE_PUMPING       3216 non-null   int64 
 12  SCOTTISH_TRANSFER          3216 non-null   int64 
 13  IFA_FLOW                   3216 non-null   int64 
 14  IFA2_FLO

In [7]:
DemandData_2022=pd.DataFrame(pd.read_csv('DemandData_2022_1690195791146.csv'))

DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('JAN'), 'Month'] = 'JAN'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('FEB'), 'Month'] = 'FEB'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('MAR'), 'Month'] = 'MAR'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('APR'), 'Month'] = 'APR'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('MAY'), 'Month'] = 'MAY'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('JUN'), 'Month'] = 'JUN'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('JUL'), 'Month'] = 'JUL'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('AUG'), 'Month'] = 'AUG'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('SEP'), 'Month'] = 'SEP'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('OCT'), 'Month'] = 'OCT'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('NOV'), 'Month'] = 'NOV'
DemandData_2022.loc[DemandData_2022['SETTLEMENT_DATE'].str.contains('DEC'), 'Month'] = 'DEC'

label="Month"
classes = DemandData_2022[label].unique().tolist()
print(f"Label classes: {classes}")

DemandData_2022[label] = DemandData_2022[label].map(classes.index)

Label classes: ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']


In [8]:
DemandData_2022.drop(['SETTLEMENT_DATE'],axis=1,inplace=True)
DemandData_2022=DemandData_2022.astype('float32')
DemandData_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SETTLEMENT_PERIOD          17520 non-null  float32
 1   ND                         17520 non-null  float32
 2   TSD                        17520 non-null  float32
 3   ENGLAND_WALES_DEMAND       17520 non-null  float32
 4   EMBEDDED_WIND_GENERATION   17520 non-null  float32
 5   EMBEDDED_WIND_CAPACITY     17520 non-null  float32
 6   EMBEDDED_SOLAR_GENERATION  17520 non-null  float32
 7   EMBEDDED_SOLAR_CAPACITY    17520 non-null  float32
 8   NON_BM_STOR                17520 non-null  float32
 9   PUMP_STORAGE_PUMPING       17520 non-null  float32
 10  IFA_FLOW                   17520 non-null  float32
 11  IFA2_FLOW                  17520 non-null  float32
 12  BRITNED_FLOW               17520 non-null  float32
 13  MOYLE_FLOW                 17520 non-null  flo

In [9]:
#DemandData_2022.groupby('SETTLEMENT_PERIOD').describe().to_excel('Demand data 2022.xlsx')

In [10]:
DemandData_2021=pd.DataFrame(pd.read_csv('DemandData_2021_1690195771550.csv'))

DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('JAN'), 'Month'] = 'JAN'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('FEB'), 'Month'] = 'FEB'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('MAR'), 'Month'] = 'MAR'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('APR'), 'Month'] = 'APR'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('MAY'), 'Month'] = 'MAY'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('JUN'), 'Month'] = 'JUN'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('JUL'), 'Month'] = 'JUL'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('AUG'), 'Month'] = 'AUG'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('SEP'), 'Month'] = 'SEP'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('OCT'), 'Month'] = 'OCT'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('NOV'), 'Month'] = 'NOV'
DemandData_2021.loc[DemandData_2021['SETTLEMENT_DATE'].str.contains('DEC'), 'Month'] = 'DEC'

label="Month"
classes = DemandData_2021[label].unique().tolist()
print(f"Label classes: {classes}")

DemandData_2021[label] = DemandData_2021[label].map(classes.index)                        

Label classes: ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']


In [11]:
DemandData_2021.drop(['SETTLEMENT_DATE'],axis=1,inplace=True)
DemandData_2021=DemandData_2021.astype('float32')
DemandData_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SETTLEMENT_PERIOD          17520 non-null  float32
 1   ND                         17520 non-null  float32
 2   TSD                        17520 non-null  float32
 3   ENGLAND_WALES_DEMAND       17520 non-null  float32
 4   EMBEDDED_WIND_GENERATION   17520 non-null  float32
 5   EMBEDDED_WIND_CAPACITY     17520 non-null  float32
 6   EMBEDDED_SOLAR_GENERATION  17520 non-null  float32
 7   EMBEDDED_SOLAR_CAPACITY    17520 non-null  float32
 8   NON_BM_STOR                17520 non-null  float32
 9   PUMP_STORAGE_PUMPING       17520 non-null  float32
 10  IFA_FLOW                   17520 non-null  float32
 11  IFA2_FLOW                  17520 non-null  float32
 12  BRITNED_FLOW               17520 non-null  float32
 13  MOYLE_FLOW                 17520 non-null  flo

In [12]:
#System_Operating_Plan=pd.DataFrame(pd.read_csv('e51f2721-00ab-4182-9cae-3c973e854aa8.csv'))

In [13]:
label="Month"

In [14]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(DemandData_2021)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

12174 examples in training, 5346 examples for testing.


In [15]:
#Normalize the data
train_mean = train_ds_pd.mean()

train_ds_pd = train_ds_pd / train_mean
test_ds_pd = test_ds_pd / train_mean

In [16]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

In [17]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

Use 4 thread(s) for training
Use /var/folders/40/rp9gqxfs1n911k0c45qn6p_c0000gn/T/tmp1fh_a0tl as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'SETTLEMENT_PERIOD': <tf.Tensor 'data:0' shape=(None,) dtype=float32>, 'ND': <tf.Tensor 'data_1:0' shape=(None,) dtype=float32>, 'TSD': <tf.Tensor 'data_2:0' shape=(None,) dtype=float32>, 'ENGLAND_WALES_DEMAND': <tf.Tensor 'data_3:0' shape=(None,) dtype=float32>, 'EMBEDDED_WIND_GENERATION': <tf.Tensor 'data_4:0' shape=(None,) dtype=float32>, 'EMBEDDED_WIND_CAPACITY': <tf.Tensor 'data_5:0' shape=(None,) dtype=float32>, 'EMBEDDED_SOLAR_GENERATION': <tf.Tensor 'data_6:0' shape=(None,) dtype=float32>, 'EMBEDDED_SOLAR_CAPACITY': <tf.Tensor 'data_7:0' shape=(None,) dtype=float32>, 'NON_BM_STOR': <tf.Tensor 'data_8:0' shape=(None,) dtype=float32>, 'PUMP_STORAGE_PUMPING': <tf.Tensor 'data_9:0' shape=(None,) dtype=float32>, 'IFA_FLOW': <tf.Tensor 'data_10:0' shape=(None,) dtype=float32>, 'IFA2_FLOW': <tf.Te

[INFO 24-03-19 09:14:47.6547 CET kernel.cc:773] Start Yggdrasil model training
[INFO 24-03-19 09:14:47.6708 CET kernel.cc:774] Collect training examples
[INFO 24-03-19 09:14:47.6710 CET kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 24-03-19 09:14:47.6786 CET kernel.cc:393] Number of batches: 13
[INFO 24-03-19 09:14:47.6787 CET kernel.cc:394] Number of examples: 12174
[INFO 24-03-19 09:14:47.6858 CET kernel.cc:794] Training dataset:
Number of records: 12174
Number of columns: 19

Number of columns by type:
	NUMERICAL: 18 (94.7368%)
	CATEGORICAL: 1 (5.26316%)

Columns:

NUMERICAL: 18 (94.7368%)
	0: "BRITNED_FLOW" NUMERICAL mean:1 min:-2.06715 max:

Model trained in 0:00:04.101088
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [18]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

6/6 [==============================] - 1s 23ms/step - loss: 0.0000e+00 - accuracy: 0.0821

loss: 0.0000
accuracy: 0.0821


In [19]:
tfdf.model_plotter.plot_model_in_colab(model_1, tree_idx=0, max_depth=4)

In [20]:
model_1.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (18):
	BRITNED_FLOW
	EAST_WEST_FLOW
	ELECLINK_FLOW
	EMBEDDED_SOLAR_CAPACITY
	EMBEDDED_SOLAR_GENERATION
	EMBEDDED_WIND_CAPACITY
	EMBEDDED_WIND_GENERATION
	ENGLAND_WALES_DEMAND
	IFA2_FLOW
	IFA_FLOW
	MOYLE_FLOW
	ND
	NEMO_FLOW
	NON_BM_STOR
	NSL_FLOW
	PUMP_STORAGE_PUMPING
	SETTLEMENT_PERIOD
	TSD

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.              "BRITNED_FLOW"  0.329822 ################
    2.                  "NSL_FLOW"  0.313233 ##############
    3.                  "IFA_FLOW"  0.192743 ######
    4.                 "IFA2_FLOW"  0.150078 ####
    5.                "MOYLE_FL

In [21]:
#DemandData_2022.groupby('Month').describe().to_excel('Demand Data 2022 stats.xlsx')

In [22]:
Demand_2021_correl=pd.DataFrame(DemandData_2021.corr(method='pearson'))
Demand_2021_correl.to_excel('Demand 2021 Pearson.xlsx')

In [23]:
#DemandData_2021.describe().transpose().to_excel('UK Demand stats 2021.xlsx')
#DemandData_2022.describe().transpose().to_excel('UK Demand stats 2022.xlsx')